## 상관분석 및 총점 계산
 - 일평균충전량을 기준으로 상관분석 수행 (MinMax 정규화 하지 않음, 즉 원본 값 그대로 사용)
 - 일평균예상충전량 및 순위 계산 [일평균예상충전량이 총점임]

### 1. 읽어들일 파일명 지정 ==> 가지고 있는 파일명으로 수정필요

In [ ]:
########### file_name 선언 ##################################
전처리데이터셋 = '남양주_전처리데이터셋.csv'
충전현황셋 = '남양주_충전량현황종합_전처리.csv' 
#############################################################

### 2. 화면에 출력되는 칼럼명을 확인한 후, 필요한 칼럼명을 복사하기
- 수정은 불필요하며 마우스로 복사만 하기

In [ ]:
import pandas as pd
df = pd.read_csv(f'{전처리데이터셋}', header=0, encoding='cp949', engine='python')
df.set_index('gid', inplace=True)
전체칼럼 = df.columns.tolist()
print ("\n여기 출력되는 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명을 마우스로 복사하세요.\n\n", 전체칼럼)

### 3. 하단에 있는 "점수칼럼"에 점수로 환산할 대상인 칼럼을 수정하도록 함 ==> 수정 필요

In [ ]:
점수칼럼 = ['전기차', '건물1', '건물2', '건물3', '관공서', '경제인구']  # 여기를 수정
print("입력한 점수컬럼명은 다음과 같음\n", 점수칼럼)

### 4. 이하 별도 언급 시까지 수정 불필요

In [ ]:
################## 1. QGIS에서 작업한 전처리 결과 CSV 읽기 ####################################################
pre_process_set = pd.read_csv(f'{전처리데이터셋}', header=0, encoding='cp949', engine='python')
pre_process_set = pre_process_set.sort_values(by='gid')

# Null 이 있을 수 있는 칼럼은 Null을 0로 변경
pre_process_set.fillna(0, inplace=True)
################## 1. QGIS에서 작업한 전처리 결과 CSV 읽기 끝###################################################


################## 2. 충전현황 파일 처리 ######################################################################
charge_summary = pd.read_csv(f'{충전현황셋}', header=0, encoding='cp949', engine='python')
charge_summary = charge_summary.sort_values(by='gid')
charge_summary.fillna(0, inplace=True)

charge_summary = charge_summary.loc[:,['gid','충전소명','일평균충전량','일평균이용수','구분']]

# 일평균충전량이 0이거나 데이터가 없는 것은 열외시킴
charge_summary.drop(index = charge_summary[charge_summary['일평균충전량']==0].index, inplace=True)   

# 상관관계를 위한 데이터셋 준비 (비정규화)
corr_prepared = charge_summary.merge(pre_process_set, on=['gid'], how='left')
################## 2. 충전현황 파일 처리 끝#####################################################################


# 상관관계 함수 미리 선언

#피어슨 상관관계 함수
def corr_p (df, col):  #col=['분석할 칼럼']
    df_ = df.loc[:,col]
    display(df_.corr(method='pearson').iloc[0:2, 2:].style.highlight_max(axis='columns'))    
    return None

#스피어만 상관관계 함수
def corr_s (df, col):  #col=['분석할 칼럼']
    df_ = df.loc[:,col]
    display("스피어만", df_.corr(method='spearman').iloc[0:2, 2:].style.highlight_max(axis='columns'))
    return None

# PV값 및 계수 확인용 함수
def pv (lm):
    pv = lm.pvalues.to_frame()
    pv.reset_index(inplace=True)
    pv.rename(columns={'index': "항목명", 0: "p-value"}, inplace=True)    
    rate = lm.params.to_frame()
    rate.reset_index(inplace=True)
    rate.rename(columns={'index': "항목명", 0: "계수"}, inplace=True)
    merged = pv.merge(rate, on='항목명', how='left')
    return (merged)

# 상관분석할 칼럼 선언
상관칼럼 = 점수칼럼.copy()
상관칼럼.insert(0, '일평균충전량')
상관칼럼.insert(1, '일평균이용수')
상관칼럼

######  상관관계 전체 보기 ########################################################################
display(corr_prepared.corr(method='pearson'))

#### 전체 충전소 ###############################################################################
display("전체 충전소", corr_p(corr_prepared, 상관칼럼))

#### 아파트 충전소 #############################################################################
apt = corr_prepared[corr_prepared['구분']=='아파트']
display("아파트충전소", corr_p(apt, 상관칼럼))

#### 비아파트 충전소 ###########################################################################
Napt = corr_prepared[corr_prepared['구분']!='아파트']
display("비아파트충전소", corr_p(Napt, 상관칼럼))


import statsmodels.formula.api as smf
######  상관 계수 분석 ########################################################################
Y = '일평균충전량'
X = '+'.join(점수칼럼)   # 상관계수에 포함시킬 항목 선언

# linear regression 그래프의 절편을 0으로 맞추는 경우 ==> -1이 의미하는 것이 절편을 영으로 하라는 옵션임
# 통계결과를 확인후 절편을 0으로 할지 아니면 유지할지는 Adj. R-squared가 높은 것으로 선택하면 됨

lm = smf.ols(formula= f'{Y} ~ -1 + {X}', data=corr_prepared).fit()
# 절편을 유지하는 경우
#lm = smf.ols(formula= f'{Y} ~ {X}', data=corr_prepared).fit()

display("통계결과", lm.summary())
coef = pv(lm)

# 상관계수 복사
rate = coef.copy()
rate.reset_index(drop=True, inplace=True)
display("상관계수", rate)

# 계수 적용할 칼럼만 추출
###### P-Value 커트라인 지정 ############
cut_over = 1
########################################
scoring = pre_process_set.copy()
scoring = scoring.sort_values(by='gid')
scoring.set_index('gid', inplace=True)
col = rate.iloc[:,0].tolist() 
scoring = scoring.loc[:,col]
scoring.reset_index(inplace=True)
scoring.sort_values(by='gid', inplace=True)

# 계수 만큼 곱하기 계산수행
for i in range(len(rate)):
    if(rate.iloc[i,1] <= cut_over):
        scoring[rate.iloc[i,0]] = scoring[rate.iloc[i,0]] * (rate.iloc[i,2])
    else:
        None   
        
                
# 총점 계산 (총점=일평균예상충전량)
scoring['예상충전량'] = scoring.sum(1)

# 등수 계산 
scoring['순위'] = scoring['예상충전량'].rank(method='dense', ascending=False).astype(int)


# 비점수칼럼을 결과에 붙이기
editing = pre_process_set.copy()
editing.set_index('gid', inplace=True)
editing.update(scoring.set_index('gid'))
editing.reset_index(inplace=True)

col_to_use = scoring.loc[:, ['gid','예상충전량','순위']]
editing = editing.merge(col_to_use, on='gid', sort=False)

# 칼럼 순서 조정 ==> 비점수칼럼 죄측, 점수캄럼 우측, 위경도는 최좌측
점수set = set(점수칼럼)
전체set = set(전체칼럼)
비점수set = 전체set - 점수set - set({'lat', 'lon'})
비점수칼럼 = list(비점수set)

col_order = ['lat', 'lon'] + 비점수칼럼 + 점수칼럼 + ['예상충전량','순위']
editing.set_index('gid', inplace=True)
editing = editing[col_order]
editing.reset_index(inplace=True)
result = editing.sort_values(by='순위')


# 최종 결과 CSV 파일쓰기
result.to_csv('결과(noscale).csv', index=False, sep=',', encoding='cp949')
final = result.copy()

display(final)

## 이하 선택 사항 - 해당없으면 Skip
### 5.1. 토지임야정보 파일이 있는 경우에만 실행 
- 파일이 없는 경우에는 실행하지 말고 Skip
- 파일이 있는 경우에는 토지격자 파일명을 갖고 있는 파일이름으로 수정 필요

In [ ]:
## 4에서 도출한 결과에 토지임야정보 추가하기
########### 추가로 file 선언 ###########################################################
#토지격자 = '남양주_국공토지격자.csv'
토지격자 = ''
#######################################################################################

### 5.2. 수정 불필요 - 토지격자셋이 있는 경우에만 실행, 없으면 Skip

In [ ]:
if (not 토지격자):
    print ('토지임야 정보가 없어서 생략합니다\n')
else:
    토지 = pd.read_csv(토지격자, header=0, encoding='cp949', engine='python')
    토지.sort_values(by='gid', inplace=True)
    gid_토지 = 토지.groupby('gid')[['gid','지목','주소','소유구분']].head(len(토지)+1)
    
    # GID 내 국유토지가 몇 건 있는지 카운팅
    국유토지_count = gid_토지.groupby('gid')[['지목']].count()
    국유토지_count.reset_index(inplace=True)
    국유토지_count.rename(columns={"지목": "국유토지"}, inplace=True)
    
    # 국유토지가 gid당 몇개 인지 결과에 붙이기
    final = final.merge(국유토지_count, on=['gid'], how='left', sort=False)
    final['국유토지'] = final['국유토지'].fillna(0).astype(int)

    final.set_index(['gid','국유토지'], inplace=True)
    final.reset_index(inplace=True)

    # 최종 결과 CSV 파일쓰기
    final.to_csv('결과(noscale).csv', index=False, sep=',', encoding='cp949')
    display(final)

### 6.1 격자센터주소 파일이 있는 경우에만 실행
- 파일이 없는 경우에는 실행하지 말고 Skip
- 파일이 있는 경우에는 격자주소셋 파일명을 갖고 있는 파일이름으로 수정 필요

In [ ]:
# 격자의 센터주소 붙이기 
########### 추가로 file 선언 ###################################################################
#격자주소셋 = '남양주_격자센터주소_전처리.csv'    # 여기 수정
격자주소셋 = '' 
###############################################################################################

### 6.2. 수정 불필요 - 격자주소셋이 있는 경우에만 실행, 없으면 Skip

In [ ]:
if (not 격자주소셋):
    print ('격자주소 정보가 없어서 생략합니다\n')
else:
    격자주소 = pd.read_csv(격자주소셋, header=0, encoding='cp949', engine='python')
    final = final.merge(격자주소, on='gid', sort=False)
    
    # 최종 결과 CSV 파일쓰기
    final.to_csv('결과(noscale).csv', index=False, sep=',', encoding='cp949')
    display(final.head(5))

### 7.1 지도로 보기 선택 사항 - 인터넷 되는 경우에만 실행
- 상단에서 도출된 결과값들 중에서 상위 3개만 지도로 보여주기
- Google Map이 보여지며, 마우스로 스크롤하면 지도 크기가 Zoom 됨
- 이를 위해서 Anaconda Promprt 창을 열고 아래 함수를 복사하여 그대로 엔터
 - pip install folium

#### 수정 필요

In [ ]:
# 보고 싶은 상위권 갯수를 입력하세요
지도보기갯수 = 3   # 보고싶은 상위권 갯수를 여기에 입력, 만약 3이면 최상위 3개만 보겠다는 의미

### 7.2 수정 불필요

In [ ]:
# 지도로 보여주기
import folium
print(f"보고자 하는 상위권 갯수는 {지도보기갯수}개 입니다.")

for i in range(지도보기갯수):
    지도보기 = folium.Map(location=[final.loc[i,'lat'],final.loc[i,'lon']], zoom_start = 16)
    folium.Marker([final.loc[i,'lat'],final.loc[i,'lon']]).add_to(지도보기)
    j = i+1
    display(f'상위 {j}번째 격자의 센터 위치 보기', 지도보기)

### 8. Box Plot 보기 선택 사항 - 수정 불필요
- 상기 선택사항 실행여부와 무관하게 9번은 실행가능함
- 값들이 몇점대에 몰려 있는지 확인해보는 Box Plot 그래프
- 맨 하단부에 몰려 있을 가능성이 높음. 이유는 절반이상이 0점이기 때문

In [ ]:
#참고) 총점 분포도
import matplotlib.pyplot as plt
plt.figure()
plt.boxplot(final['예상충전량'])
plt.show()

끝